#### Bayessche Statistik Interaktiv

Der Satz von Bayes wird hier anhand eines Besipiels gezeigt. Einzelne Wahrscheinlichkeiten können interaktiv geändert werden wodurch die Änderungen am Endergebnnis sofort sichtbar sind.

Erläuterungen zum Beispiel:

- Als Beispiel soll ein medizinischer Test gewahlt werden. Herauszufinden gilt (a posteriori Wahrscheinlichkeit):
 * Ist ein Patient erkrankt, wenn der Test positiv anschlägt?

Interaktive Eingaben:
- Wenn der Patient erkrankt ist, schlaegt der Test mit einer Wahrscheinlichkeit von 0.xx an. (voreingestellt: 0.99)
- in x-Fällen ist der Test positiv, auch wenn der Patient nicht erkrankt ist (voreingestellt: 0.005)
- Prior Information, d.h. wie weit ist die Krankheit verbreitet (voreingestellt: 0.0001)

Daraus werden zunaechst die bedingten Wahrscheinlichkeiten berechnet:
- P(krank)
- P(nicht krank)
- P(test positiv|erkrankt)
- P(test positiv | nicht erkrankt)

Und dann mittels des Bayesschen Theorems P(krank|test positiv) - was wir wissen wollen.


Theorem von Bayes:
Für zwei Ereignisse A und B, für B ≠ 0, lautet das Satz von Bayes:

$$ {P(A|B)} = \frac{P(B|A) \bullet P(A)}{P(B)}$$

P(A | B) ist die (bedingte) Wahrscheinlichkeit des Ereignisses A unter der Bedingung, dass B eingetreten ist. P(B | A) ist die (bedingte) Wahrscheinlichkeit des Ereignisses B unter der Bedingung, dass A eingetreten ist
P(A) ist die Wahrscheinlichkeit (Anfangswahrscheinlichkeit) für das Eintreten des Ereignisses A
P(B) ist die Wahrscheinlichkeit (Anfangswahrscheinlichkeit) für das Eintreten des Ereignisses B
Anfangswahrscheinlichkeit meint, dass ein Ereignis unabhängig von einem anderen betrachtet wird.

In [ ]:
#resourcen

import ipywidgets as widgets
import numpy as np

In [ ]:
# interaktive Schalter

# Nachweissicherheit wenn Patient erkrankt
selectionValues = np.arange(0.0001, 1, 0.0001)
pB_A = widgets.SelectionSlider( options=selectionValues, value=0.99, orientation='horizontal',\
                               readout=True, layout={'width': 'max-content'})
# Anteil positver Tests bei gesundem Patienten
pB_Anot = widgets.SelectionSlider( options=selectionValues, value=0.005, orientation='horizontal',\
                               readout=True, layout={'width': 'max-content'}, style = {'description_width': 'initial'})
#
pA = widgets.SelectionSlider( options=selectionValues, value=selectionValues[0], orientation='horizontal',\
                               readout=True, layout={'width': '20%'})
#
pVerbreitung = widgets.HBox(
    [
        widgets.Label(value="Verbreitung der Krankheit:"),
        pA
    ]
)
#
pNachweis = widgets.VBox(
    [
        widgets.Label(
            value='Nachweissicherheit des Tests wenn der patient erkrankt ist:'),
        pB_A        
    ]
)
#
pFehler = widgets.VBox(
    [
        widgets.Label(value='Fehlerrate des Tests wenn der Patient nicht erkrankt ist:'),
        pB_Anot
    ]
)
#
style = {'description_width': 'initial'}
outpriori = widgets.VBox(
[
    # P(erkrankt) = pA
    widgets.Box([widgets.Label(value="P(erkrankt)", layout={'width': '30%'}),
                widgets.FloatText(description = "", value = pA.value ,\
                                  layout={'width': '20%'}, style=style)]
                ),
    # P(nicht erkrankt) = 1-pA
    widgets.Box([widgets.Label(value="P(nicht erkrankt)", layout={'width': '30%'}),
                 widgets.FloatText(value = ((1 - pA.value)),\
                                   layout ={'width': '20%'}, style=style)]
               ),
    # P(test_positiv|erkrankt) = pB_A (pNachweissicherheit)
    widgets.Box([widgets.Label(value= "P(test_positiv|erkrankt)", layout={'width': '30%'}),
                 widgets.FloatText(value = '%1.6f' % (pB_A.value),\
                                   layout={'width': '20%'}, style=style)]
               ),
    # P(test_positiv|nicht erkrankt) = pB_Anot (positiver Test bei gesundem Patienten)
    widgets.Box([widgets.Label(value="P(test_positiv|nicht erkrankt)", layout={'width': '30%'}),
                 widgets.FloatText(value = '%1.6f' %(pB_Anot.value),\
                                   layout={'width': '20%'}, style=style)]
               )
])
# P(erkrankt|test_positiv) = [pA * pNachweissicherheit] / [pA * pNachweissicherheit + pA_not * pFehlerrate] {= pB über gesamtwahrscheinlichkeit}
outposteriori = widgets.VBox(
[
    widgets.Label(value="Wahrscheinlichkeit dass ein Patient erkrankt ist, wenn der Test positiv anschlägt:"),
    widgets.FloatText(value = '%1.6f' % (pB_A.value * pA.value / \
                                                (pB_A.value * pA.value + pB_Anot.value * (1 - pA.value))),\
                      description="", layout={'width': '30%'})
])
#
BtnReset = widgets.Button(description="reset", button_style='warning')
#

In [ ]:
# update-Funktion die beim Ändern eines Schalters ausgefuehrt wird
#
def update_view(*args):
    """ Wahrscheinlichkeit, dass ein Patient erkrankt ist,\
        wenn der Test positiv anschlägt, mit den gegebenen Parametern neu berechnen.
                                       _ 
             p(B|A) = 0,99         p(B|A) = 0.05
            /                   _ /
        p(A)                  p(A)
            \  _                  \  _ _
             p(B|A) = 0,01         p(B|A) = 0.95 
             
        P(erkrankt) = pA
        P(nicht erkrankt) = 1-pA
        P(test_positiv|erkrankt) = pB_A (default: 0.99)
            {P(test_negativ|erkrankt) = 1- pB_A (default: 1 - 0.99)} nicht dargestellt 
        P(test_positiv|nicht erkrankt) = pB_Anot (default: 0.05) 
            {P(test_negativ|nicht erkrankt) = 1 - pB_Anot (default: 1- 0.05)} nicht dargestellt
        P(erkrankt|test_positiv) = [pB_A] * pA / [pB_A*pA + pB_Anot*1-pA] {=> P(B) über gesamtwahrscheinlichkeit gerechnet}
        """
    outpriori.children[0].children[1].value = pA.value
    outpriori.children[1].children[1].value = (1 - pA.value)
    outpriori.children[2].children[1].value = '%1.6f' % (pB_A.value)
    outpriori.children[3].children[1].value = '%1.6f' %(pB_Anot.value)
    #
    # Extremwerte abfangen
    if (pB_Anot.value == 0):
            outposteriori.children[1].value = 0.0
            return
    if (pA.value == 1):
            outposteriori.children[1].value = 1
            return
    outposteriori.children[1].value = '%1.6f' % ((pB_A.value * pA.value) / \
                                                 (pB_A.value * pA.value + pB_Anot.value * (1-pA.value)))
    #

In [ ]:
# Einstellungen auf default-Werte zuruecksetzen
#
def reset(*btn):
    pA.value = 0.0001
    pB_A.value = 0.99
    pB_Anot.value= 0.005
    update_view()
    

In [ ]:
# update-Funktion hinzufügen
#
pA.observe(update_view, 'value')
pB_A.observe(update_view, 'value')
pB_Anot.observe(update_view, 'value')
BtnReset.on_click(reset)
#
# Schalter anordnen
#
widgets.VBox([widgets.VBox([pVerbreitung, pNachweis, pFehler]),
             outpriori,
             outposteriori,
             BtnReset])